In [1]:
import pandas as pd
import configparser

# Read the value from the config.ini file
config = configparser.ConfigParser()
config.read('config.ini')
apr_data_file = config.get('apriori', 'apr_data_file')
# Read the csv file
# input_file should be the path to the merge csv file
try:
    input_file = apr_data_file
    df = pd.read_csv(input_file)
except FileNotFoundError as e:
    print(e)
    # input_file = r"C:\Users\Trieu Pham\OneDrive - BTM Global Consulting\Projects\data csv\outputFile\data.csv"
    # df = pd.read_csv(input_file)
df.dropna(inplace=True)

In [2]:
df

,StoreID,Date,InvoiceID,ItemName,Barcode,Category Name,SubCategory Name
0,VN0001,2023-01-01,VN0001010101230001,DASANI Nước khoáng 510ml x1 Chai,8935049500544,Bottled Water,Mineral water
1,VN0001,2023-01-01,VN0001010101230002,SATORI Nước tinh khiết 500ml x 1 Chai,8938512632025,Bottled Water,Purified water
2,VN0001,2023-01-01,VN0001010101230003,Celano Kem Bánh Cá Trân Châu Dừa Tắc 70g * 1 cây,8936011773416,Packaged Ice Cream/Novelties,Other Ice Cream
3,VN0001,2023-01-01,VN0001010101230003,LAYS WAVY Khoai tây bò Texas 63g x gói,8936079121761,Potato Chips,Potato Snacks
4,VN0001,2023-01-01,VN0001010101230003,MILO Kem que Magma 55g x1 Cây,8850453017528,Packaged Ice Cream/Novelties,Stick
...,...,...,...,...,...,...,...
21308937,VN0243,2023-04-30,VN0243023004230048,THTM Không đường 180ml x1 Hộp,8935217400058,Whole Milk,UHT Milk
21308939,VN0243,2023-04-30,VN0243023004230049,Cơm nắm kim chi phô mai,2270102000361,Rice Offsite,Onigiri
21308940,VN0243,2023-04-30,VN0243023004230049,FUZE TEA+ Trà Chanh với Sả 450ml x 1 Chai,8935049501190,Tea,Sugar Tea
21308941,VN0243,2023-04-30,VN0243023004230049,FUZETEA+ Trà đào hạt chia 450ml x 1 Chai,8935049500698,Tea,Sugar Tea


In [19]:
import configparser

config = configparser.ConfigParser()
config.read('config.ini')

filter_value = config.getint('filter', 'filter_value')



In [20]:
# List of unique stores
stores = df['StoreID'].unique()

# Sum number of stores
len(stores)


219

In [21]:

if filter_value == 1:
    df_filter = df[['InvoiceID','Category Name']]
    df_filter = df_filter[df_filter.groupby('InvoiceID')['Category Name'].transform('size') > 1]
    dataset_str = df_filter.groupby('InvoiceID').agg({
    'Category Name': lambda x: ','.join(str(i) for i in x)
    }).reset_index()
    
if filter_value == 2:
    df_filter = df[['InvoiceID','SubCategory Name']]
    df_filter = df_filter[df_filter.groupby('InvoiceID')['SubCategory Name'].transform('size') > 1]
    dataset_str = df_filter.groupby('InvoiceID').agg({
    'SubCategory Name': lambda x: ','.join(str(i) for i in x)
    }).reset_index()
else:
    print('Filter value is not correct')

Filter value is not correct


In [22]:
dataset_str

,InvoiceID,Category Name
0,VN0001010101230001,"Bottled Water,Bottled Water"
1,VN0001010101230002,"Bottled Water,Bottled Water"
2,VN0001010101230003,"Packaged Ice Cream/Novelties,Potato Chips,Pack..."
3,VN0001010101230004,"Liquor,Cold Beverages,Add-on item Cat,Liquor,C..."
4,VN0001010101230005,"Sports Drinks,Sports Drinks,Sports Drinks,Spor..."
...,...,...
5270664,VN9996023101230486,"Sports Drinks,Sports Drinks,Energy Drinks"
5270665,VN9996023101230488,"Rice Offsite,Cold Beverages,Energy Drinks"
5270666,VN9996023101230489,"Potato Chips,Cold Beverages,Liquor"
5270667,VN9996023101230492,"Cigarettes Category,Cold Beverages"


In [25]:
test = dataset_str[dataset_str['Category Name'].str.count(',') < 1] 
test

,InvoiceID,Category Name


In [26]:
# Get StoreID from InvoiceID
dataset_str['StoreID'] = dataset_str['InvoiceID'].map(lambda x: x[:6])
# Convert the date string to a datetime object and extract the month
dataset_str['Month'] = dataset_str['InvoiceID'].str[10:12].map(lambda x: int(x[:2]))

In [27]:
def apriopri(df, filter_value, min_support, metric):
    from mlxtend.preprocessing import TransactionEncoder
    from mlxtend.frequent_patterns import apriori, association_rules
    if filter_value == 1:
        data = df[['Category Name']]
    elif filter_value == 2:
        data = df[['SubCategory Name']]
    # create a TransactionEncoder object
    data = data.values.tolist()

    data = [x[0].split(',') for x in data]

    te = TransactionEncoder()

    # transform the data into a list of lists
    transactions = te.fit(data).transform(data)
    transactions = pd.DataFrame(transactions, columns=te.columns_)

    # Calculate frequent itemsets using Apriori algorithm
    frequent_itemsets = apriori(transactions, min_support=min_support, use_colnames=True)

    # Calculate rules
    rules = association_rules(frequent_itemsets, metric=metric, min_threshold=0)
    
    # Add a new column to the rules DataFrame to calculate the conviction
    rules['conviction'] = (1 - rules['support']) / (1 - rules['confidence'])
    return rules


In [15]:
min_support = config.getfloat('apriori', 'min_support')
metric = config.get('apriori', 'metric')

# rules = apriopri(dataset_str, filter_value, min_support, metric)

In [17]:
# List of unique months
months = dataset_str['Month'].unique()

# Sum number of months
len(months)
stores


array(['VN0001', 'VN0002', 'VN0003', 'VN0005', 'VN0006', 'VN0007',
       'VN0008', 'VN0009', 'VN0012', 'VN0014', 'VN0015', 'VN0017',
       'VN0019', 'VN0020', 'VN0021', 'VN0022', 'VN0025', 'VN0027',
       'VN0028', 'VN0030', 'VN0031', 'VN0032', 'VN0033', 'VN0034',
       'VN0036', 'VN0037', 'VN0038', 'VN0041', 'VN0042', 'VN0043',
       'VN0048', 'VN0049', 'VN0050', 'VN0051', 'VN0053', 'VN0054',
       'VN0056', 'VN0061', 'VN0062', 'VN0064', 'VN0065', 'VN0066',
       'VN0067', 'VN0068', 'VN0069', 'VN0070', 'VN0072', 'VN0074',
       'VN0075', 'VN0076', 'VN0077', 'VN0079', 'VN0081', 'VN0082',
       'VN0083', 'VN0084', 'VN0086', 'VN0087', 'VN0088', 'VN0089',
       'VN0090', 'VN0091', 'VN0092', 'VN0093', 'VN0095', 'VN0096',
       'VN0097', 'VN0098', 'VN0099', 'VN0100', 'VN0101', 'VN0102',
       'VN0104', 'VN0105', 'VN0107', 'VN0108', 'VN0109', 'VN0110',
       'VN0111', 'VN0112', 'VN0113', 'VN0115', 'VN0116', 'VN0117',
       'VN0118', 'VN0119', 'VN0120', 'VN0121', 'VN0122', 'VN01

In [41]:
import os
fail_store = []
apr_results_file = config.get('apriori', 'apr_results_file')
# Clear all files in the folder
for file in os.listdir(apr_results_file):
    file_path = os.path.join(apr_results_file, file)
    try:
        if os.path.isfile(file_path):
            os.unlink(file_path)
    except Exception as e:
        print(e)
for i in range(1,len(months)+1):
    month_df = dataset_str[dataset_str['Month'] == i]
    for store in stores:
        store_df = month_df[month_df['StoreID'] == store]
        try:
            rules = apriopri(store_df, filter_value, min_support, metric)
            rules['StoreID'] = store
            rules['Month'] = i
            # change rules columns order
            rules = rules[['StoreID', 'Month', 'antecedents', 'consequents', 'support', 'lift', 'confidence', 'conviction']]
            # Rename columns
            rules.columns = ['StoreID', 'Month', 'Trigger', 'Dependence', 'Support', 'Lift', 'Confidence', 'Conviction']
            if filter_value == 1:
                file_output = os.path.join(apr_results_file, 'Result_M' + str(i) + '_' + 'Cate_Filter' +  str(min_support) + '.csv')
            elif filter_value == 2:
                file_output = os.path.join(apr_results_file, 'Result_M' + str(i) + '_' + 'SubCate_Filter' +  str(min_support) + '.csv')
            else:
                print('Filter value is not correct')
            # Check if the file already exists, if not create a new file
            if not os.path.isfile(file_output):
                with open(file_output, 'w') as f:
                    f.write('')
            with open(file_output, 'r') as f:
                first_line = f.readline().strip()
                # Check if the first line contains the expected column names
                has_headers = 'support' in first_line
            rules.to_csv(file_output, index=False, mode='a', header= not has_headers)
        except Exception as e:
            fail_store.append('Store ' + store + ' Month ' + str(i) + ' ' + str(e))
# Make empty error.log file
open('error.log', 'w').close()

# Write the failed stores to error.log file
with open('error.log', 'w') as f:
    for item in fail_store:
        f.write("%s\n" % item)